In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl
    import dolfinx
else:
    try:
        import dolfinx
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenicsx-install.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
        import ufl
        import dolfinx

In [ ]:
assert dolfinx.__version__ == "0.1.1.0"

In [ ]:
from mpi4py import MPI

In [ ]:
mesh = dolfinx.UnitIntervalMesh(MPI.COMM_WORLD, 3)

In [ ]:
V = dolfinx.FunctionSpace(mesh, ("CG", 1))

In [ ]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

In [ ]:
dx = ufl.dx

In [ ]:
import numpy as np

In [ ]:
f = dolfinx.Function(V)
dim_V = f.vector.local_size
assert dim_V == 4
f.vector[:] = np.arange(1, dim_V + 1)

In [ ]:
a = u * v * dx
F = f * v * dx

In [ ]:
A = dolfinx.fem.assemble_matrix(a)
b = dolfinx.fem.assemble_vector(F)

In [ ]:
A.assemble()

In [ ]:
solution = dolfinx.Function(V)

In [ ]:
from petsc4py import PETSc

In [ ]:
for package in (None, "mumps", "superlu", "superlu_dist"):
    ksp = PETSc.KSP().create()
    ksp.setOperators(A)
    ksp.setType("preonly")
    ksp.getPC().setType("lu")
    if package is not None:
        ksp.getPC().setFactorSolverType(package)
    ksp.setFromOptions()
    ksp.solve(b, solution.vector)
    solution.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    assert np.allclose(solution.vector.getArray(), np.arange(1, dim_V + 1))

In [ ]:
grad = ufl.grad
inner = ufl.inner

In [ ]:
k = inner(grad(u), grad(v)) * dx

In [ ]:
K = dolfinx.fem.assemble_matrix(k)
K.assemble()

In [ ]:
expected = (0, 10.8, 54, 108)

In [ ]:
from slepc4py import SLEPc

In [ ]:
for package in (None, "mumps", "superlu", "superlu_dist"):
    eps = SLEPc.EPS().create()
    eps.setOperators(K, A)
    eps.setProblemType(SLEPc.EPS.ProblemType.GHEP)
    eps.setWhichEigenpairs(SLEPc.EPS.Which.TARGET_REAL)
    eps.setTarget(1)
    st = eps.getST()
    st.setType(SLEPc.ST.Type.SINVERT)
    st.setShift(1)
    ksp = st.getKSP()
    ksp.setType("preonly")
    pc = ksp.getPC()
    pc.setType("lu")
    if package is not None:
        pc.setFactorSolverType(package)
    eps.solve()
    assert eps.getConverged() == len(expected)
    for (i, eig_i_ex) in enumerate(expected):
        eig_i = eps.getEigenvalue(i)
        assert np.isclose(eig_i.real, eig_i_ex)
        assert np.isclose(eig_i.imag, 0)